In [ ]:
import numpy as np
from scipy import constants 
import matplotlib.pyplot as plt
import os

def fftconvolve(ax,ay,axis=-1):
    nex=ax.shape[axis]+ay.shape[axis]-1
    return np.fft.ifft(np.fft.fft(ax,n=nex,axis=axis)*np.fft.fft(ay,n=nex,axis=axis),axis=axis)

# def DBF(gaz,gele,wl,a0,element_pos,we):
#     k=np.pi*2/wl
#     # thetaxy = np.arctan2(np.tan(thetay),np.tan(thetax))
#     # phixy = np.arctan(np.sqrt(np.tan(thetay)**2+np.tan(thetax)**2))
#     ar = np.array([np.sin(gele)*np.sin(gaz), np.sin(gele)*np.cos(gaz), np.cos(gele)])
#     ar = ar-a0
#     # print(ar)
#     Exy=np.zeros((*we.shape[:2],*ar.shape[1:]))
#     for ie in range(element_pos.shape[0]):
#         de = element_pos[ie,:]
#         element_phase = np.sum(ar*de[:,np.newaxis,np.newaxis],axis=0)
#         # print(element_phase*180/np.pi)|
#         xe = we[:,:,ie]
#         Exy = Exy + xe[:,:,np.newaxis,np.newaxis]*np.exp(1j*k*element_phase)[np.newaxis,np.newaxis,:,:]
#     return Exy

def DBF(gaz,gele,wl,a0,element_pos,we):
    k=np.pi*2/wl
    # thetaxy = np.arctan2(np.tan(thetay),np.tan(thetax))
    # phixy = np.arctan(np.sqrt(np.tan(thetay)**2+np.tan(thetax)**2))
    ar = np.array([np.sin(gele)*np.sin(gaz), np.sin(gele)*np.cos(gaz), np.cos(gele)])
    ar = ar-a0
    # print(ar)
    Exy=np.zeros((*we.shape[:2],*ar.shape[1:]))
    for ie in range(element_pos.shape[1]):
        de = element_pos[:,ie,:]
        element_phase = np.sum(ar[np.newaxis,:,:,:]*de[:,:,np.newaxis,np.newaxis],axis=1)
        # print(element_phase*180/np.pi)|
        xe = we[:,:,ie]
        Exy = Exy + xe[:,:,np.newaxis,np.newaxis]*np.exp(1j*k*element_phase)[:,np.newaxis,:,:]
    return Exy

def element_rotate(element_pos,orientation):
    # this rotation is clockwise not math degree
    out_pos = np.zeros((orientation.size,*element_pos.shape))
    for ik, displace in enumerate(orientation):
        deltadeg = displace*np.pi/180
        Mr = np.array([[np.cos(deltadeg),np.sin(deltadeg),0],[-np.sin(deltadeg),np.cos(deltadeg),0],[0,0,1]])
        out_pos[ik,:,:] = np.transpose(np.matmul(Mr,np.transpose(element_pos)))
    return out_pos

In [ ]:
TS = 290        # noise temperature K
npulse = 64    # number of pulses in sequence
nfft = 1024      # zero padding in fft
Pt = 100        # peak power    W
G = 100         # Gain
F0 = 10.35e9       # carrier frequency Hz
wavelength = constants.c/F0     # radar wavelength  m
B=20e6
ts = 2e-8/3       # sample rate   s
PRF = 600e3     # PRF   Hz
PRT=1/PRF
Tx = 1/PRF      # pulse width
Va=constants.c*PRF/4/F0
Lx=Tx/ts
chirp = B/Tx
Ra=constants.c/PRF/2    # maximum unambiguous range
# print(Ra)

de = wavelength/2
xx,yy = np.meshgrid(np.arange(-3.5,3.6,1),np.arange(-1.5,1.6,1))
element_loc0 = np.array([xx.flatten(),yy.flatten(),np.zeros(xx.size)]).transpose()*de

theta0 = 0 * np.pi/180  # azimuth y->x
phi0 = 0 * np.pi/180    #zenith
a0 = np.array([np.sin(phi0)*np.sin(theta0), np.sin(phi0)*np.cos(theta0), np.cos(phi0)])[:,np.newaxis,np.newaxis]

txp = np.arange(0,Tx,ts)    # discrete time of pulse
# create pulse window
# tmpwd = np.hanning(txp.size//10)
wd = np.ones(txp.size)
# wd[:tmpwd.size//2] = tmpwd[:tmpwd.size//2]
# wd[-tmpwd.size//2:] = tmpwd[-tmpwd.size//2:]
tx = np.zeros(txp.shape,dtype=np.complex128)
tx[:tx.size//2] = Pt*wd[:tx.size//2]*np.exp((-np.pi*B*txp[:tx.size//2]+2*np.pi*chirp*txp[:tx.size//2]**2)*1j) # transmit waveform
txpl=txp[tx.size//2:]-Tx/2
tx[tx.size//2:] = Pt*wd[tx.size//2:]*np.exp((np.pi*B*txpl-2*np.pi*chirp*txpl**2)*1j)
# tf = np.arange(1e-5*ts,(2*rswath/constants.c)//ts*ts+1e-5*ts,ts)  # fast time
tf = txp
fast_time = tf[np.newaxis,:]
tm = np.arange(npulse)*PRT
tm = tm[:,np.newaxis]
Faxe=np.arange(-1/2/ts,1/2/ts,1/ts/tf.size)  # frequency after fft
Faxe = Faxe[np.newaxis,:]

In [ ]:
# def fire_pulse(element_loct,target_p):
#     total_rx = np.zeros((npulse,tf.size,int(element_loct.shape[0])),dtype=tx.dtype)
#     for Rtar,Vtar,rcs, azt, elt in zip(*target_p):
#         tau_0 = 2*Rtar/constants.c # first pulse target time delay
#         FD = 2*Vtar/wavelength  # target doppler
#         ARtar = constants.c*(tau_0-np.floor(tau_0/PRT)*PRT)/2 # apperent range of target
#         Radar_c = np.sqrt(G**2*wavelength**2*rcs/(4*np.pi)**3)
#         # *np.exp(2j*np.pi*np.random.random(1)) # radar constant without scale of range
#         rr = constants.c*(tau_0)/2       # range
#         Radar_c = Radar_c/rr**2     # return power
    
#         rx=np.zeros(tf.shape,dtype=tx.dtype)       # received signal
#         rx[:tx.size]=tx
    
#         tau_m = 2*(Rtar-tm*Vtar)/constants.c-tref
#         rx=np.tile(rx,(npulse,1))
#         Radar_c=Radar_c
#         frx=np.fft.fftshift(np.fft.fft(rx,axis=1),axes=1) 
#             # frequency content of received signal
#         dfrx=frx*np.exp(-2j*np.pi*Faxe*tau_m)   # time delay in frequency domain
    
#         rx=np.fft.ifft(np.fft.fftshift(dfrx,axes=1))*Radar_c
#         rx=rx*np.exp(2j*np.pi*FD*fast_time)*np.exp(-2j*np.pi*F0*tau_m)
#         az = azt*np.pi/180
#         ele = elt*np.pi/180
#         targetRvector = np.array([np.sin(ele)*np.sin(az),np.sin(ele)*np.cos(az),np.cos(ele)])[np.newaxis,:]
#         dl = np.sum(element_loct*targetRvector,axis=1)[np.newaxis,np.newaxis,:]
#         total_rx=total_rx+rx[:,:,np.newaxis]*np.exp(-2j*np.pi*dl/wavelength)
#     return total_rx
def fire_pulse(element_loct,target_p):
    total_rx = np.zeros((npulse,tf.size,int(element_loct.shape[1])),dtype=tx.dtype)
    for Rtar,Vtar,rcs, azt, elt in zip(*target_p):
        tau_0 = 2*Rtar/constants.c # first pulse target time delay
        FD = 2*Vtar/wavelength  # target doppler
        ARtar = constants.c*(tau_0-np.floor(tau_0/PRT)*PRT)/2 # apperent range of target
        Radar_c = np.sqrt(G**2*wavelength**2*rcs/(4*np.pi)**3)
        # *np.exp(2j*np.pi*np.random.random(1)) # radar constant without scale of range
        rr = constants.c*(tau_0)/2       # range
        Radar_c = Radar_c/rr**2     # return power
    
        rx=np.zeros(tf.shape,dtype=tx.dtype)       # received signal
        rx[:tx.size]=tx
    
        tau_m = 2*(Rtar-tm*Vtar)/constants.c
        rx=np.tile(rx,(npulse,1))
        Radar_c=Radar_c
        frx=np.fft.fftshift(np.fft.fft(rx,axis=1),axes=1) 
            # frequency content of received signal
        dfrx=frx*np.exp(-2j*np.pi*Faxe*tau_m)   # time delay in frequency domain
    
        rx=np.fft.ifft(np.fft.fftshift(dfrx,axes=1))*Radar_c
        rx=rx*np.exp(2j*np.pi*FD*fast_time)*np.exp(-2j*np.pi*F0*tau_m)
        az = azt*np.pi/180
        ele = elt*np.pi/180
        targetRvector = np.array([np.sin(ele)*np.sin(az),np.sin(ele)*np.cos(az),np.cos(ele)])[np.newaxis,np.newaxis,:]
        dl = np.sum(element_loct*targetRvector,axis=2)[:,np.newaxis,:]
        total_rx=total_rx+rx[:,:,np.newaxis]*np.exp(-2j*np.pi*dl/wavelength)
    return total_rx

In [ ]:
# Rtar_list = [2.95e3,3e3,3.05e3,3.05e3]        # target range

# Rtar_list = [3e3,3e3,3e3,3e3]        # target range
# Atar_list = [90,     90,  90, 90]              # target azimuth
# Etar_list = [0,     0,  30,-10]              # target elevation
# Vtar_list = [-20,0,15,-30]               # fix target radial velocity
# rcs_list = [1000,1000,1000,1000]         # radar cross section   m^2

Rtar_list = [30,30]        # target range
Atar_list = [ 60,90]              # target azimuth
Etar_list = [-20,30]              # target elevation
Vtar_list = [60,-40]               # fix target radial velocity
rcs_list = [10,10]         # radar cross section   m^2

# Rtar_list = [30]        # target range
# Atar_list = [90]              # target azimuth
# Etar_list = [30]              # target elevation
# Vtar_list = [0]               # fix target radial velocity
# rcs_list = [10]         # radar cross section   m^2

target_para = [Rtar_list, Vtar_list, rcs_list, Atar_list, Etar_list]

In [ ]:
baseline_azimuth = 0
element_loct = element_rotate(element_loc0,baseline_azimuth*np.ones(npulse))
rx = fire_pulse(element_loct,target_para)

In [ ]:
# plt.plot(constants.c*txp/2,np.angle(tx),'r')
# plt.plot(constants.c*txp/2,np.real(rx[0,:,1]*np.conj(tx)),'b')
# rx[0,:,1]
# plt.grid()

## Single target lag-1 estimator using beat frequency

In [ ]:
Fr = np.arange(-1/2/ts,1/2/ts,1/ts/nfft)
crxu = rx[:,:tx.shape[0]//2,:]*np.conj(tx[np.newaxis,:tx.shape[0]//2,np.newaxis])
crxd = rx[:,-tx.shape[0]//2:,:]*np.conj(tx[np.newaxis,tx.shape[0]//2:,np.newaxis])
Fbu = -np.angle(np.mean((crxu[:,1:,:]*np.conj(crxu[:,:-1,:])),axis=(0,1)))/ts/np.pi/2
Fbd = np.angle(np.mean((crxd[:,1:,:]*np.conj(crxd[:,:-1,:])),axis=(0,1)))/ts/np.pi/2
crxu = np.mean(crxu,axis=0)
crxd = np.mean(crxd,axis=0)
plt.plot(-Fr,np.abs(np.fft.fftshift(np.fft.fft(crxu,n=nfft,axis=0),axes=0)),'b-')
plt.plot(Fr,np.abs(np.fft.fftshift(np.fft.fft(crxd,n=nfft,axis=0),axes=0)),'r-')
plt.xlim(-1.2*B,1.2*B)
# plt.plot(-Fr,np.transpose(np.abs(np.fft.fftshift(np.fft.fft(rx[:,:tx.shape[0]//2,1]*np.conj(tx[np.newaxis,:tx.shape[0]//2]),n=nfft,axis=1)))),'b-')
# plt.plot(Fr,np.transpose(np.abs(np.fft.fftshift(np.fft.fft(rx[:,tx.shape[0]//2:,1]*np.conj(tx[np.newaxis,tx.shape[0]//2:]),n=nfft,axis=1)))),'r-')
# plt.xlim(-1e8*0.25,1e8*0.25)
# plt.xlim(-5e7,5e7)
plt.figure()
rFr = Fr/(2*chirp)*constants.c/2
plt.plot(-rFr,np.abs(np.fft.fftshift(np.fft.fft(crxu,n=nfft,axis=0),axes=0)),'b-')
plt.plot(rFr,np.abs(np.fft.fftshift(np.fft.fft(crxd,n=nfft,axis=0),axes=0)),'r-')
plt.xlim(-0.6*Ra,0.6*Ra)
# print(Fbu)
# print()
# print(Fbd)

# print()
# print((Fbd+Fbu)/4/(2*chirp)*constants.c)

# print()
# print((Fbd-Fbu)/4*wavelength)
# # plt.plot(-rFr,np.transpose(np.abs(np.fft.fftshift(np.fft.fft(rx[:,:tx.shape[0]//2,1]*np.conj(tx[np.newaxis,:tx.shape[0]//2]),n=nfft,axis=1),axes=1))),'b-')
# # plt.plot(rFr,np.transpose(np.abs(np.fft.fftshift(np.fft.fft(rx[:,tx.shape[0]//2:,1]*np.conj(tx[np.newaxis,tx.shape[0]//2:]),n=nfft,axis=1),axes=1))),'r-')

## Range Doppler map

In [ ]:
Fr = np.arange(-1/2/ts,1/2/ts,1/ts/nfft)
crxu = rx[:,:tx.shape[0]//2,:]*np.conj(tx[np.newaxis,:tx.shape[0]//2,np.newaxis])
crxd = rx[:,-tx.shape[0]//2:,:]*np.conj(tx[np.newaxis,tx.shape[0]//2:,np.newaxis])
crxu = np.mean(crxu,axis=2)
crxd = np.mean(crxd,axis=2)
nv=int(2**(np.log(crxu.shape[0])//np.log(2)+1))
rdm=np.abs(np.fft.fftshift(np.fft.fft(np.fft.fftshift(np.fft.fft(crxu,n=nv,axis=0),axes=0),n=nfft,axis=1),axes=1))
Fv=np.arange(-PRF/2,PRF/2,PRF/nv)
plt.pcolormesh(Fv/2*wavelength,-rFr,np.transpose(rdm))
plt.xlim(-100,100)
plt.ylim(0,400)

In [ ]:
def all_digital(element_loct,rx):
    crx = rx[:,:tx.shape[0]//2,:]*np.conj(tx[np.newaxis,:tx.shape[0]//2,np.newaxis])
    Eloc = DBF(thetaaz,thetaele,wavelength,a0,element_loct,crx)
    Eloc = np.fft.fftshift(np.fft.fft(Eloc,n=nfft,axis=1),axes=1)
    Beam = np.mean(np.abs(Eloc)**2,axis=0)
    Fr = np.arange(-1/2/ts,1/2/ts,1/ts/nfft)
    r = Fr/(2*chirp)*constants.c/2
    plt.pcolormesh(np.squeeze(thetax),np.squeeze(thetay),np.squeeze(Beam[np.abs(r-30)==np.min(np.abs(r-30)),:,:]))
    plt.ylim(-40,40)
    plt.xlim(-40,40)
    plt.colorbar()

In [ ]:
def subarray(element_loct,rx):
    subarray_rx = np.sum(rx.reshape((rx.shape[0],rx.shape[1],4,8)), axis=2)
    subarray_loc0 = np.mean(element_loct.reshape((element_loct.shape[0],4,8,3)), axis=1)
    all_digital(subarray_loc0,subarray_rx)

### beamforming @ $\theta_x$, $\theta_y$ 

In [ ]:
beamx = np.arange(-45,46,1.5) * np.pi/180
beamy = np.arange(-45,46,1.5) * np.pi/180
thetax, thetay = np.meshgrid(beamx, beamy)
thetaele = np.arccos(np.sqrt(1-np.sin(thetax)**2-np.sin(thetay)**2))
thetaaz = np.arctan2(np.sin(thetax),np.sin(thetay))
thetax = thetax*180/np.pi
thetay = thetay*180/np.pi

In [ ]:
# baseline_azimuth = 150
# element_loct = element_rotate(element_loc0,baseline_azimuth*np.ones(npulse))
element_loct = element_rotate(element_loc0,np.arange(0,64))
rx = fire_pulse(element_loct,target_para)
plt.figure()
all_digital(element_loct,rx)
plt.plot(np.array([-90,90])*np.cos(baseline_azimuth*np.pi/180),-np.array([-90,90])*np.sin(baseline_azimuth*np.pi/180),'r')
plt.axis([-40,40,-40,40])
plt.figure()
subarray(element_loct,rx)

element_loct = element_rotate(element_loc0,np.arange(0,64*3,3))
rx = fire_pulse(element_loct,target_para)
plt.figure()
all_digital(element_loct,rx)
plt.plot(np.array([-90,90])*np.cos(baseline_azimuth*np.pi/180),-np.array([-90,90])*np.sin(baseline_azimuth*np.pi/180),'r')
plt.axis([-40,40,-40,40])
plt.figure()
subarray(element_loct,rx)